# SCRIPT POUR RÉCUPÉRER DES DONNÉES SUR BLOOMBERG (Exécutable uniquement sur un PC disposant de l'outil Bloomberg)

Afin d'exécuter le code, cliquez sur Run => Run All Cells.

### 1- Installation des Modules

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Modules for the buttons and widgets 
import ipywidgets as widgets
from ipywidgets import Button, Layout, Text,  HBox
from ipywidgets import Dropdown
from ipywidgets import Output
from IPython.display import display

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Modules pour Bloomberg
import blpapi
from xbbg import blp

### 2- Exportation des données historiques des titres à partir de Bloomberg (Exécutable uniquement sur un PC disposant de l'outil Bloomberg)

* On utilise la fonction bdh et on définit les tickers (Symboles comme définit sur Bloomberg ex: SNTS BC EQUITY) qui nous intéressent, la start_date ou end_date sous le format YYYY-MM-DD et la période (Yearly 'Y', Weekly 'W' ou Daily 'D').

* Days= 'A' signifie "récupérer tous les jours y compris les week-ends" et Fill='P' signifie "si vous n'avez pas de données pour un jour donné, remplir avec les données disponibles par le passé".

* Si vous ne souhaitez pas définir une date de début pour avoir toutes les données disponibles sans restriction supprimer dans le code la start date.

* Si vous ne souhaitez pas prendre en compte les week-ends, supprimer Days='A' dans le code.

* Si vous ne souhaitez pas appeler tous les titres, supprimer les tickers dont vous n'avez pas besoin.

* Si vous n'avez besoin que du prix de clôture, il suffira de supprimer certaines des mentions dans fields (flds). 

* 'EQY_DPS' permet d'avoir le dividende.

* 'TOT_RETURN_INDEX_GROSS_DVDS' est un champ Bloomberg principalement utilisé pour calculer le rendement total d'un indice sur une période donnée. Cela comprend à la fois l'appréciation du prix et le réinvestissement des dividendes.

* 'EQY_TOTAL_RETURN_INDEX' est un champ Bloomberg principalement utilisé pour calculer le rendement total d'une action sur une période donnée. Cela comprend à la fois l'appréciation du prix et le réinvestissement des dividendes.

* Si vous ne souhaitez pas remplir les données non disponibles pour un jour donné par celles de la veille, supprimer Fill='P'.

* Afin de connaitre les caractéristiques de la fonction bdh, mettez le curseur aprés la fonction et cliquer sur shift+tab.

* Les fichiers Excel des données recherchées sont diponibles dans le file browser, il suffira de les télécharger.

* Les ratios présentaient sont le P/E, D/Y, ROE, ROA, ROC, ROIC, Payout ratio, EBITDA Margin, Operating Margin.

* Pour extraire des données sur excel, il suffit d'utiliser le script nom du fichier à exporter.to_excel('nom.xlsx',index=True). le fichier xlsx est créé dans le dossier python, un clique droit et il sera téléchargé.

In [2]:
# Dropdown pour la sélection des titres
data_selection = {
    "Tous les Secteurs": ['CABC BC EQUITY','FTSC BC EQUITY','NEIC BC EQUITY','NTLC BC EQUITY','SEMC BC EQUITY','SIVC BC EQUITY','SLBC BC EQUITY','SMBC BC EQUITY','STBC BC EQUITY','TTRC BC EQUITY','UNLC BC EQUITY','UNXC BC EQUITY','CIEC BC EQUITY','ONTBF BC EQUITY','ORAC BC EQUITY','SDCC BC EQUITY','SNTS BC EQUITY','BICC BC EQUITY','BOAB BC EQUITY','BOABF BC EQUITY','BOAC BC EQUITY','BOAM BC EQUITY','BOAN BC EQUITY','BOAS BC EQUITY','CBIBF BC EQUITY','ECOC BC EQUITY','ETIT BC EQUITY','NSBC BC EQUITY','ORGT BC EQUITY','SAFC BC EQUITY','SGBC BC EQUITY','SIBC BC EQUITY','SDSC BC EQUITY','SVOC BC EQUITY','PALC BC EQUITY','SCRC BC EQUITY','SICC BC EQUITY','SOGC BC EQUITY','SPHC BC EQUITY','ABJC BC EQUITY','BNBC BC EQUITY','CFAC BC EQUITY','PRSC BC EQUITY','SHEC BC EQUITY','TTLC BC EQUITY','TTLS BC EQUITY','STAC BC EQUITY'],
    "Secteur Finance": ['BOAB BC EQUITY','BOABF BC EQUITY','BOAC BC EQUITY','BOAM BC EQUITY','BOAN BC EQUITY','BOAS BC EQUITY','BICC BC EQUITY','CBIBF BC EQUITY','ECOC BC EQUITY', 'ETIT BC EQUITY', 'NSBC BC EQUITY','ORGT BC EQUITY','SAFC BC EQUITY','SGBC BC EQUITY','SIBC BC EQUITY'],
    "Secteur Services Publics": ['CIEC BC EQUITY', 'ONTBF BC EQUITY','ORAC BC EQUITY','SDCC BC EQUITY','SNTS BC EQUITY'],
    "Secteur Industrie":['SIVC BC EQUITY','SEMC BC EQUITY','FTSC BC EQUITY','NEIC BC EQUITY','NTLC BC EQUITY','CABC BC EQUITY','STBC BC EQUIT','SMBC BC EQUITY','SLBC BC EQUITY','UNLC BC EQUITY','UNXC BC EQUITY'],
    "Secteur Agriculture": ['PALC BC EQUITY','SPHC BC EQUITY','SICC BC EQUITY','SOGC BC EQUITY','SCRC BC EQUITY'],
    "Secteur Distribution": ['BNBC BC EQUITY','CFAC BC EQUITY','ABJC BC EQUITY','TTLC BC EQUITY','TTLS BC EQUITY','SHEC BC EQUITY','PRSC BC EQUITY'],
    "Secteur Transport": ['SDSC BC EQUITY','SVOC BC EQUITY'],
    "Autres Secteurs": ['STAC BC EQUITY'],
    "Indices":['ICXCOMP Index','ICXAGR Index','ICXIND Index','ICX10 Index','ICXAUT Index','ICXDIS Index','ICXFIN Index','ICXSPU Index','ICXTRP Index'],
}

data_options = list(data_selection.keys()) 
data_dropdown = Dropdown(
    description="Secteur:",
    options=data_options,
    value=data_options[0], 
)


# Dropdown pour la sélection des dates
date_selection = {
    "1998": '1998-09-16',
    "2000": '2000-01-01',
    "2005": '2005-01-01', 
    "2010": '2010-01-01', 
    "2017": '2017-01-01',
    "2018": '2018-01-01',
    "2019": '2019-01-01',
    "2020": '2020-01-01',
    "2021": '2021-01-01',
    "2022": '2022-01-01', 
    "2023": '2023-01-01', 
    "2024": '2024-01-01', 
    "2025": '2025-01-01', 
    "2026": '2026-01-01', 
    "2027": '2027-01-01', 
    "2028": '2028-01-01',
    "2029": '2029-01-01', 
    "2030": '2030-01-01', 
    "2031": '2031-01-01',
    "2032": '2032-01-01', 
    "2033": '2033-01-01', 
    "2034": '2034-01-01',     
}


date_options = list(date_selection.keys()) 
date_dropdown = Dropdown(
    description="Start Date:",
    options=date_options,
    value=date_options[0], 
)

# Dropdown pour la sélection des fréquences
freq_selection = {
    "Journalière":'D',
    "Hebdomadaire": 'W', 
    "Mensuelle": 'M', 
    "Annuelle": 'Y', 
}


freq_options = list(freq_selection.keys()) 
freq_dropdown = Dropdown(
    description="Périodicité:",
    options=freq_options,
    value=freq_options[0], 
)


# Définition des fonctions de calcul

def last_price():
    Last_price=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['last_price'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    Last_toexcel=Last_price.to_excel('Dernier_Cours.xlsx',index=True)
    print(Last_price)

last_price_button = widgets.Button(description="Dernier Cours",button_style='info')
last_price_button.layout.width = '240px'
last_price_button.layout.height = '40px'
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        last_price()

last_price_button.on_click(on_button_clicked)

def high_low():
    high_low=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['open','high','low'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    high_low_toexcel=high_low.to_excel('Open_High_Low.xlsx',index=True)
    print(high_low)

high_low_button = widgets.Button(description="Open / High_Low",button_style='info')
high_low_button.layout.width = '240px'
high_low_button.layout.height = '40px'
output1 = widgets.Output()

def on_button_clicked1(b):
    with output1:
        output1.clear_output()
        high_low()

high_low_button.on_click(on_button_clicked1)


def vol_mktcap():
    vol_mktcap=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['volume','cur_mkt_cap'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    vol_mktcap_toexcel=vol_mktcap.to_excel('Volume_Mktcap.xlsx',index=True)
    print(vol_mktcap)

vol_mktcap_button = widgets.Button(description="Volume / Capitalisation Boursière",button_style='info')
vol_mktcap_button.layout.width = '240px'
vol_mktcap_button.layout.height = '40px'
output2 = widgets.Output()

def on_button_clicked2(b):
    with output2:
        output2.clear_output()
        vol_mktcap()

vol_mktcap_button.on_click(on_button_clicked2)


def div():
    div=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['EQY_DPS'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    div_toexcel= div.to_excel('Dividendes.xlsx',index=True)
    print(div)

div_button = widgets.Button(description="Dividendes",button_style='info')
div_button.layout.width = '240px'
div_button.layout.height = '40px'
output3 = widgets.Output()

def on_button_clicked3(b):
    with output3:
        output3.clear_output()
        div()

div_button.on_click(on_button_clicked3)

def tot_ret_ind():
    tot_ret_ind=blp.bdh(tickers=data_selection["Indices"],flds=['TOT_RETURN_INDEX_GROSS_DVDS'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    totret_ind_toexcel= tot_ret_ind.to_excel('Total Return Indice.xlsx',index=True)
    print(tot_ret_ind)

tot_ret_button = widgets.Button(description="Total Return Indices",button_style='info')
tot_ret_button.layout.width = '240px'
tot_ret_button.layout.height = '40px'
output4 = widgets.Output()

def on_button_clicked4(b):
    with output4:
        output4.clear_output()
        tot_ret_ind()

tot_ret_button.on_click(on_button_clicked4)

def tot_ret_eqy():
    tot_ret_eqy=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['EQY_TOTAL_RETURN_INDEX'],start_date=date_selection[date_dropdown.value],Per=freq_selection[freq_dropdown.value])
    totret_eqy_toexcel= tot_ret_eqy.to_excel('Total Return Actions.xlsx',index=True)
    print(tot_ret_eqy)

tot_ret_eqy_button = widgets.Button(description="Total Return Actions",button_style='info')
tot_ret_eqy_button.layout.width = '240px'
tot_ret_eqy_button.layout.height = '40px'
output5 = widgets.Output()

def on_button_clicked5(b):
    with output5:
        output5.clear_output()
        tot_ret_eqy()

tot_ret_eqy_button.on_click(on_button_clicked5)

def ratio():
    ratio=blp.bdh(tickers=data_selection[data_dropdown.value],flds=['PE_RATIO','DIVIDEND_YIELD','RETURN_COM_EQY','RETURN_ON_ASSET','RETURN_ON_CAP','RETURN_ON_INV_CAPITAL','DVD_PAYOUT_RATIO','EBITDA_TO_REVENUE','OPER_MARGIN'],start_date=date_selection[date_dropdown.value],Per=freq_selection["Annuelle"])
    ratio_toexcel=ratio.to_excel('Ratios.xlsx',index=True)
    print(ratio)

ratio_button = widgets.Button(description="Ratios",button_style='info')
ratio_button.layout.width = '240px'
ratio_button.layout.height = '40px'

output6 = widgets.Output()

def on_button_clicked6(b):
    with output6:
        output6.clear_output()
        ratio()

ratio_button.on_click(on_button_clicked6)

# Creation d'un HBox layout 
hbox = HBox(children=[last_price_button,high_low_button,vol_mktcap_button,div_button,tot_ret_button,tot_ret_eqy_button,ratio_button])

display(data_dropdown,date_dropdown,freq_dropdown,hbox,output,output1,output2,output3,output4,output5,output6)   


Dropdown(description='Secteur:', options=('Tous les Secteurs', 'Secteur Finance', 'Secteur Services Publics', …

Dropdown(description='Start Date:', options=('1998', '2000', '2005', '2010', '2017', '2018', '2019', '2020', '…

Dropdown(description='Périodicité:', options=('Journalière', 'Hebdomadaire', 'Mensuelle', 'Annuelle'), value='…

Output()

Output()

Output()

Output()

Output()

Output()

Output()